In [4]:
import pandas as pd
import joblib
from flask import Flask, render_template, request
from threading import Thread

# Inisialisasi aplikasi Flask
app = Flask(__name__)

# Memuat model dan encoder yang telah disimpan
loaded_model = joblib.load('best_rf_model_Random_Forest.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Membaca data mata kuliah dari file Excel
file_path = 'Data MK dan Nilai.xlsx'  # Ganti dengan path file Anda
data = pd.read_excel(file_path)

# Menyaring hanya kolom 'kode_mk' dan 'nama_mk' dari data
mata_kuliah = data[['kode_mk', 'nama_mk']]

@app.route('/')
def index():
    # Mengambil daftar mata kuliah untuk dropdown
    mata_kuliah_list = mata_kuliah['nama_mk'].tolist()
    return render_template('index.html', mata_kuliah_list=mata_kuliah_list)

@app.route('/prediksi', methods=['POST'])
def prediksi():
    # Mendapatkan nama mata kuliah yang dipilih oleh pengguna
    nama_mk_input = request.form['mata_kuliah']
    
    # Mendapatkan kode_mk terkait nama_mk
    kode_mk_input = mata_kuliah[mata_kuliah['nama_mk'] == nama_mk_input]['kode_mk'].values[0]

    # Mengencode kode mata kuliah
    if kode_mk_input in label_encoder.classes_:
        kode_mk_encoded = label_encoder.transform([kode_mk_input])[0]
    else:
        kode_mk_encoded = int(label_encoder.transform(label_encoder.classes_).mean())
        print(f"Kode mata kuliah '{kode_mk_input}' tidak ditemukan, menggunakan nilai rata-rata untuk prediksi.")
    
    # Membuat DataFrame untuk prediksi
    new_data = pd.DataFrame({'kode_mk_encoded': [kode_mk_encoded]})

    # Melakukan prediksi
    prediction = loaded_model.predict(new_data)

    # Menampilkan hasil prediksi
    result = f"Klasifikasi tingkat kesulitan untuk kode mata kuliah '{kode_mk_input}': {prediction[0]}"
    return render_template('index.html', result=result, mata_kuliah_list=mata_kuliah['nama_mk'].tolist())

# Fungsi untuk menjalankan Flask di dalam thread
def run_flask():
    app.run(debug=True, host='0.0.0.0', port=2001, use_reloader=False)

# Menjalankan Flask di dalam kernel Jupyter
flask_thread = Thread(target=run_flask)
flask_thread.start()


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.4.2 when using version 1.5.2. This might

 * Serving Flask app '__main__'
 * Debug mode: on
